# Classic Doom Speed Demo Data Analzer

This Jupyter Notebook will take as input a csv file made according the specifications laid out in the README.md file of this repository and, hopefully, generate something interesting.

This was all done in pursuit of learning, so this notebook will be structured as though the reader is also a student (with some prior Python and statistics experience). I don't expect many people will be doing serious data work with Doom speed demos, but in case you would like, making some small changes to this notebook should be simple work.

In [29]:
import numpy as np
import pandas as pd

In [30]:
df = pd.read_csv('./examples/uv-speed-data.csv')
levels = [
    # Starport Levels
    'ENTRYWAY',
    'UNDERHALLS',
    'THE GANTLET',
    'THE FOCUS',
    'THE WASTE TUNNELS',
    'THE CRUSHER',
    # Hellish Outpost Levels
    'DEAD SIMPLE',
    'TRICKS AND TRAPS',
    'THE PIT',
    'REFUELING BASE',
    'CIRCLE OF DEATH',
    # City Levels
    'THE FACTORY',
    'DOWNTOWN',
    'THE INMOST DENS',
    'INDUSTRIAL ZONE',
    'SUBURBS',
    'TENEMENTS',
    'THE COURTYARD',
    'THE CITADEL',
    'GOTCHA',
    # Inside Hell Levels
    'NIRVANA',
    'THE CATACOMBS',
    'BARRELS O FUN',
    'BARRELS D FUN', # OCR mistake.
    'THE CHASM',
    'BLOODFALLS',
    'BLDODFALLS', # OCR mistake.
    'ABANDONED MINES',
    'MONSTER CONDO',
    'THE SPIRIT WORLD',
    'THE LIVING END',
    'ICON OF SIN',
    # Secret Levels
    'WOLFENSTEIN',
    'GROSSE'
]

In [31]:
df.shape

(2701, 3)

This data looks like quite a mess, due largely to the screen melt effect that happens between levels. Unfortunately, there is no way to disable it short of modifying the source code, so its effects simply must be dealt with after the fact.

In [34]:
df[10:30]

,armor,health,level
10,4,2 5,JEZM V F
11,NaN,NaN,NaN
12,0,1 3,NaN
13,0,100,NaN
14,0,100,
15,0,100,NaN
16,0,100,NaN
17,0,100,LWQ
18,0,100,NaN
19,0,100,


# Munging Steps

1. Remove all rows with any `NaN` values.
1. Replace with `NaN` any `level` values that are not in the `LEVELS` list.
1. Back fill the `level` column.
1. Delete any rows with invalid values for `health` or `armor`.
1. Find a way to fix incongruent values near level transitions.

I will begin by cleaning up the `level` column. I do this because I know that Doom does not simultaneously show the level name and hud data at the same time. Therefore, if I can use tidy `level` data to help differentiate between good and bad `health` and `armor` data.

My first step is to clear anyting out of the `level` column that does not match an element in the `LEVELS` list.

In [20]:
df.level[~df.level.isin(levels)] = np.nan
df

,armor,health,level
0,,,NaN
1,,,NaN
2,,,NaN
3,,,NaN
4,,,NaN
5,,,NaN
6,,,NaN
7,,,NaN
8,,,NaN
9,4,2 5,NaN


This looks much better already, but I still have more work to do. Certainly `health` and `armor` values that share a row with a valid `level` value are wrong, but what about the ones before and after? During these rows, the screen was in the process of the screen melt effect, leading to bad OCR results.

My inelegant, but easy and fast solution is to spread values in the `level` column forward and and back for a few cells, which will make selecting and deleting these rows later eaier to do.

# HERE IS WHERE I STOPPED

In [21]:

df.level = df.level.fillna(method='backfill', limit=2)
df.level = df.level.fillna(method='ffill', limit=4)
df

,armor,health,level
0,,,NaN
1,,,NaN
2,,,NaN
3,,,NaN
4,,,NaN
5,,,NaN
6,,,NaN
7,,,NaN
8,,,NaN
9,4,2 5,NaN


In [22]:
# Clear health and armor cells adjacent to valid level cells.
df.health[df.level.isin(levels)] = np.nan
df.armor[df.level.isin(levels)] = np.nan
df

,armor,health,level
0,,,NaN
1,,,NaN
2,,,NaN
3,,,NaN
4,,,NaN
5,,,NaN
6,,,NaN
7,,,NaN
8,,,NaN
9,4,2 5,NaN


In [23]:
# Backfill the level column.
df.level = df.level.fillna(method='backfill')
df

,armor,health,level
0,,,ENTRYWAY
1,,,ENTRYWAY
2,,,ENTRYWAY
3,,,ENTRYWAY
4,,,ENTRYWAY
5,,,ENTRYWAY
6,,,ENTRYWAY
7,,,ENTRYWAY
8,,,ENTRYWAY
9,4,2 5,ENTRYWAY


In [24]:
# Convert health and armor to Float64 Series.
df.health = pd.to_numeric(df.health, errors='coerce')
df.armor = pd.to_numeric(df.armor, errors='coerce')
df

,armor,health,level
0,NaN,NaN,ENTRYWAY
1,NaN,NaN,ENTRYWAY
2,NaN,NaN,ENTRYWAY
3,NaN,NaN,ENTRYWAY
4,NaN,NaN,ENTRYWAY
5,NaN,NaN,ENTRYWAY
6,NaN,NaN,ENTRYWAY
7,NaN,NaN,ENTRYWAY
8,NaN,NaN,ENTRYWAY
9,4.0,NaN,ENTRYWAY


In [25]:
# Drop any rows with a NaN value.
df.dropna(inplace=True)
df

,armor,health,level
13,0.0,100.0,ENTRYWAY
14,0.0,100.0,ENTRYWAY
15,0.0,100.0,ENTRYWAY
16,0.0,100.0,ENTRYWAY
17,0.0,100.0,ENTRYWAY
18,0.0,100.0,ENTRYWAY
19,0.0,100.0,ENTRYWAY
20,0.0,100.0,ENTRYWAY
21,0.0,100.0,ENTRYWAY
22,0.0,100.0,ENTRYWAY


In [26]:
# Drop health and armor values that are out of range.
df = df[(df.health > 0) & (df.health <= 200)]
df = df[(df.armor >= 0) & (df.armor <= 200)]
df

,armor,health,level
13,0.0,100.0,ENTRYWAY
14,0.0,100.0,ENTRYWAY
15,0.0,100.0,ENTRYWAY
16,0.0,100.0,ENTRYWAY
17,0.0,100.0,ENTRYWAY
18,0.0,100.0,ENTRYWAY
19,0.0,100.0,ENTRYWAY
20,0.0,100.0,ENTRYWAY
21,0.0,100.0,ENTRYWAY
22,0.0,100.0,ENTRYWAY
